In [2]:

from __future__ import absolute_import, division, print_function, unicode_literals
import numpy as np 
import pandas as pd

import xgboost as xgb
from xgboost import XGBClassifier

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import Imputer
from sklearn.model_selection import GridSearchCV

import altair as alt
#alt.renderers.enable('notebook')
alt.renderers.enable('colab')


RendererRegistry.enable('colab')

In [3]:
#reading data
orders=pd.read_csv("Colab Notebooks/instacart-market-basket-analysis/orders.csv")

departments=pd.read_csv("Colab Notebooks/instacart-market-basket-analysis/departments.csv")
aisles=pd.read_csv("Colab Notebooks/instacart-market-basket-analysis/aisles.csv")
products=pd.read_csv("Colab Notebooks/instacart-market-basket-analysis/products.csv")
order_product_train=pd.read_csv("Colab Notebooks/instacart-market-basket-analysis/order_products__train.csv")
order_product_prior=pd.read_csv("Colab Notebooks/instacart-market-basket-analysis/order_products__prior.csv")
print("[Out]:Files Successfully Loaded in frames")

[Out]:Files Successfully Loaded in frames


In [4]:
#showing data
#printing 5 top row from each file
print("\norder:")
print(orders.head(5))
print("\ndepartments:")
print(departments.head(5))

print("\naisles:")

print(aisles.head(5))

print("\nproducts:")
print(products.head(5))

print("\n order_product_train")

print(order_product_train.head(5))
print("\n order_product_pripor:")

print(order_product_prior.head(5))



order:
   order_id  user_id  ... order_hour_of_day  days_since_prior_order
0   2539329        1  ...                 8                     NaN
1   2398795        1  ...                 7                    15.0
2    473747        1  ...                12                    21.0
3   2254736        1  ...                 7                    29.0
4    431534        1  ...                15                    28.0

[5 rows x 7 columns]

departments:
   department_id department
0              1     frozen
1              2      other
2              3     bakery
3              4    produce
4              5    alcohol

aisles:
   aisle_id                       aisle
0         1       prepared soups salads
1         2           specialty cheeses
2         3         energy granola bars
3         4               instant foods
4         5  marinades meat preparation

products:
   product_id  ... department_id
0           1  ...            19
1           2  ...            13
2           3  ...   

In [5]:
#shape of dataframes
def shape_info():
    print("[OUT]:orders shape:",orders.shape)
    print("[OUT]:departments shape:",departments.shape)
    print("[OUT]:aisles shape",aisles.shape)
    print("[OUT]:products shape",aisles.shape)
    print("[OUT]:order product train shape",order_product_train.shape)
    print("[OUT]:order product prior shape",order_product_prior.shape)
shape_info()

[OUT]:orders shape: (3421083, 7)
[OUT]:departments shape: (21, 2)
[OUT]:aisles shape (134, 2)
[OUT]:products shape (134, 2)
[OUT]:order product train shape (1384617, 4)
[OUT]:order product prior shape (32434489, 4)


In [6]:
#data preprocessing 

def missing_summary():
    order_miss=orders.isnull().sum()
    print("[OUT]:Missing data summary for orders: \n",pd.concat([order_miss,order_miss/orders.isnull().count()],keys=['Total Miss','Percentage Miss'],axis=1))
    
    depa_miss=departments.isnull().sum()
    print("\n\n[OUT]:Missing data summary for departments:\n",pd.concat([depa_miss,depa_miss/departments.isnull().count()],keys=['Total Miss','Percentage Miss'],axis=1))
    
    aisle_miss=aisles.isnull().sum()
    print("\n\n[OUT]:Missing data summary for aisles:\n",pd.concat([aisle_miss,aisle_miss/aisles.isnull().count()],keys=['Total Miss','Percentage Miss'],axis=1))
    
    prod_miss=products.isnull().sum()
    print("\n\n[OUT]:Missing values summary for departments:\n",pd.concat([prod_miss,prod_miss/products.isnull().count()],keys=['Total Miss','Percentage Miss'],axis=1))
    
    op_train_miss=order_product_train.isnull().sum()
    print("\n\n[OUT]:Missing values summary for departments:\n",pd.concat([op_train_miss,op_train_miss/order_product_train.isnull().count()],keys=['Total Miss','Percentage Miss'],axis=1))
    
    op_prior_miss=order_product_prior.isnull().sum()
    print("\n\n[OUT]:Missing values summary for departments:\n",pd.concat([op_prior_miss,op_prior_miss/order_product_prior.isnull().count()],keys=['Total Miss','Percentage Miss'],axis=1))
    
    
missing_summary()

[OUT]:Missing data summary for orders: 
                         Total Miss  Percentage Miss
order_id                         0         0.000000
user_id                          0         0.000000
eval_set                         0         0.000000
order_number                     0         0.000000
order_dow                        0         0.000000
order_hour_of_day                0         0.000000
days_since_prior_order      206209         0.060276


[OUT]:Missing data summary for departments:
                Total Miss  Percentage Miss
department_id           0              0.0
department              0              0.0


[OUT]:Missing data summary for aisles:
           Total Miss  Percentage Miss
aisle_id           0              0.0
aisle              0              0.0


[OUT]:Missing values summary for departments:
                Total Miss  Percentage Miss
product_id              0              0.0
product_name            0              0.0
aisle_id                0        

In [7]:
#define preprocessing

def clean(orders):
    print("\n\n[OUT]: Cleaning Starts:")
    orders=orders[orders['days_since_prior_order'].notnull()]
    print("\n[OUT]: Cleaning Done.")
clean(orders)

print("\n[OUT]:Shapes After Clean:\n")
shape_info()




[OUT]: Cleaning Starts:

[OUT]: Cleaning Done.

[OUT]:Shapes After Clean:

[OUT]:orders shape: (3421083, 7)
[OUT]:departments shape: (21, 2)
[OUT]:aisles shape (134, 2)
[OUT]:products shape (134, 2)
[OUT]:order product train shape (1384617, 4)
[OUT]:order product prior shape (32434489, 4)


In [8]:
#plot Count of Prior and Train data in orders

distribution_count=orders['eval_set'].value_counts().rename_axis('Label').reset_index(name='Order_Count')
print("\n[Out]: Count of Prior and Train data in orders : \n",distribution_count)
alt.Chart(distribution_count).mark_bar().encode(x=alt.X('Label',sort=alt.EncodingSortField(
            field="Order_Count",  # The field to use for the sort
            order="descending"  # The order to sort in
       )),y='Order_Count',color='Label')
  


[Out]: Count of Prior and Train data in orders : 
    Label  Order_Count
0  prior      3214874
1  train       131209
2   test        75000


alt.Chart(...)

In [9]:
print("[OUT]: Count of Order Per Day of Week")

day_of_week_count=orders['order_dow'].value_counts().rename_axis('Week_Day').reset_index(name='Order_Count')
week=['Sunday','Monday','Tuesday','Wednesday','Thursday','Friday','Saturday']
day_of_week_count['Day_Name']=week

alt.Chart(day_of_week_count).mark_bar().encode(x=alt.X('Day_Name',sort=alt.EncodingSortField(
            field="Order_Count",  # The field to use for the sort
            order="descending"  # The order to sort in
       )), y='Order_Count',color='Day_Name')


[OUT]: Count of Order Per Day of Week


alt.Chart(...)

In [10]:
print("[OUT]:Count of Orders per Hour of Day")

hour_day=orders['order_hour_of_day'].value_counts().rename_axis('Hour_of_Day').reset_index(name='Order_Count')

alt.Chart(hour_day).mark_bar().encode(x='Hour_of_Day',y='Order_Count',color='Order_Count')

[OUT]:Count of Orders per Hour of Day


alt.Chart(...)

In [11]:
print("[OUT]: Order Count on Sunday per hour of the day")

week_data=orders[orders['order_dow']==0]
week_day_hour=week_data['order_hour_of_day'].value_counts().rename_axis('Hour_of_Day').reset_index(name='Order_Count')

alt.Chart(week_day_hour).mark_bar().encode(x='Hour_of_Day',y='Order_Count',color='Order_Count')

[OUT]: Order Count on Sunday per hour of the day


alt.Chart(...)

In [12]:
print("[OUT]: Order Count on Monday per hour of the day")

week_data=orders[orders['order_dow']==1]
week_day_hour=week_data['order_hour_of_day'].value_counts().rename_axis('Hour_of_Day').reset_index(name='Order_Count')

alt.Chart(week_day_hour).mark_bar().encode(x='Hour_of_Day',y='Order_Count',color='Order_Count')

[OUT]: Order Count on Monday per hour of the day


alt.Chart(...)

In [13]:
print("[OUT]: Order Count on Tuesday per hour of the day")
week_data=orders[orders['order_dow']==2]
week_day_hour=week_data['order_hour_of_day'].value_counts().rename_axis('Hour_of_Day').reset_index(name='Order_Count')
alt.Chart(week_day_hour).mark_bar().encode(x='Hour_of_Day',y='Order_Count',color='Order_Count')

[OUT]: Order Count on Tuesday per hour of the day


alt.Chart(...)

In [14]:
print("[OUT]: Order Count on Wednesday per hour of the day")
week_data=orders[orders['order_dow']==3]
week_day_hour=week_data['order_hour_of_day'].value_counts().rename_axis('Hour_of_Day').reset_index(name='Order_Count')
alt.Chart(week_day_hour).mark_bar().encode(x='Hour_of_Day',y='Order_Count',color='Order_Count')

[OUT]: Order Count on Wednesday per hour of the day


alt.Chart(...)

In [15]:
print("[OUT]: Order Count on Thursday per hour of the day")

week_data=orders[orders['order_dow']==4]
week_day_hour=week_data['order_hour_of_day'].value_counts().rename_axis('Hour_of_Day').reset_index(name='Order_Count')

alt.Chart(week_day_hour).mark_bar().encode(x='Hour_of_Day',y='Order_Count',color='Order_Count')

[OUT]: Order Count on Thursday per hour of the day


alt.Chart(...)

In [16]:
print("[OUT]: Order Count on Friday per hour of the day")

week_data=orders[orders['order_dow']==5]
week_day_hour=week_data['order_hour_of_day'].value_counts().rename_axis('Hour_of_Day').reset_index(name='Order_Count')

alt.Chart(week_day_hour).mark_bar().encode(x='Hour_of_Day',y='Order_Count',color='Order_Count')

[OUT]: Order Count on Friday per hour of the day


alt.Chart(...)

In [17]:
print("[OUT]: Order Count on Saturday per hour of the day")

week_data=orders[orders['order_dow']==6]
week_day_hour=week_data['order_hour_of_day'].value_counts().rename_axis('Hour_of_Day').reset_index(name='Order_Count')

alt.Chart(week_day_hour).mark_bar().encode(x='Hour_of_Day',y='Order_Count',color='Order_Count')

[OUT]: Order Count on Saturday per hour of the day


alt.Chart(...)

In [18]:

print("[OUT]:Heatmap of per day of week per hour order count")
day_hour_order = orders.groupby(["order_dow", "order_hour_of_day"])["order_number"].aggregate("count").reset_index()
        
#day_hour_order= day_hour_order.pivot('order_dow', 'order_hour_of_day', 'order_number')



alt.Chart(day_hour_order).mark_rect().encode(
    x=alt.X('order_hour_of_day:N', title="Hour of the day"),
    y=alt.Y('order_dow:N', title="day_of_week"),
    color=alt.Color('order_number:Q', title="Order_Count")
)



[OUT]:Heatmap of per day of week per hour order count


alt.Chart(...)

In [19]:
print("[OUT]:Count of order w.r.t. Days Prior to order")

prior_day = orders['days_since_prior_order'].value_counts().rename_axis('Days_Since_Prior_Order').reset_index(name='Order_Count')


alt.Chart(prior_day).mark_bar().encode(x='Days_Since_Prior_Order',y='Order_Count',color='Order_Count')


[OUT]:Count of order w.r.t. Days Prior to order


alt.Chart(...)

In [0]:
#merging for plotting 
merg_order_prior = pd.merge(order_product_prior, orders, on='order_id', how='left')
merged_all= pd.merge(order_product_prior, products, on='product_id', how='left')
merged_all= pd.merge(merged_all, aisles, on='aisle_id', how='left')
merged_all=pd.merge(merged_all, departments, on='department_id', how='left')

In [21]:
print("[OUT]:Count of orders which were reordered")

reord_count=order_product_prior['reordered'].value_counts().rename_axis('Label').reset_index(name='Order_Count')
reord_count['label_name']=['reordered','not reordered']

alt.Chart(reord_count).mark_bar().encode(x=alt.X('label_name',sort=alt.EncodingSortField(
            field="Order_Count",  # The field to use for the sort
            order="descending"  # The order to sort in
       )), y='Order_Count',color='label_name')

[OUT]:Count of orders which were reordered


alt.Chart(...)

In [22]:
print("[OUT]:Count of reorders in day of week")

aggr_dow_reord = merg_order_prior.groupby(["order_dow"])["reordered"].aggregate("count").reset_index()
aggr_dow_reord['Day_of_Week']=['Sunday','Monday','Tuesday','Wednesday','Thursday','Friday','Saturday']
aggr_dow_reord.columns=['order_dow','Count of reordered','Day_of_Week']

alt.Chart(aggr_dow_reord).mark_bar().encode(x=alt.X('Day_of_Week',sort=alt.EncodingSortField(
            field="Count of reordered",  # The field to use for the sort
            order="descending"  # The order to sort in
       )), y='Count of reordered',color='Day_of_Week')

[OUT]:Count of reorders in day of week


alt.Chart(...)

In [23]:
print("[OUT]: Count of reorders per hour in a day")

aggr_hur_reord = merg_order_prior.groupby(["order_hour_of_day"])["reordered"].aggregate("count").reset_index()
aggr_hur_reord.columns=['order_hour_of_day','Count_of_reordered']



alt.Chart(aggr_hur_reord).mark_bar().encode(x=alt.X('order_hour_of_day',sort=alt.EncodingSortField(
            field="Count_of_reordered",  # The field to use for the sort
            order="descending"  # The order to sort in
       )), y='Count_of_reordered',color='Count_of_reordered')

[OUT]: Count of reorders per hour in a day


alt.Chart(...)

In [24]:
print("[OUT]: Count of reorders w.r.t Product Name")

reord_products = merged_all['product_name'].value_counts().rename_axis('Product Name').reset_index(name="Reorder Count").head(10)



alt.Chart(reord_products).mark_bar().encode(x=alt.X('Product Name',sort=alt.EncodingSortField(
            field="Product Name",  # The field to use for the sort
            order="descending"  # The order to sort in
       )), y='Reorder Count',color='Product Name')

[OUT]: Count of reorders w.r.t Product Name


alt.Chart(...)

In [25]:
print("[OUT]: Count of reorders w.r.t Aisle")

reord_aisle = merged_all['aisle'].value_counts().rename_axis('Aisle Name').reset_index(name="Reorder Count").head(10)

alt.Chart(reord_aisle).mark_bar().encode(x=alt.X('Aisle Name',sort=alt.EncodingSortField(
            field="Aisle Name",  # The field to use for the sort
            order="descending"  # The order to sort in
       )), y='Reorder Count',color='Aisle Name')

[OUT]: Count of reorders w.r.t Aisle


alt.Chart(...)

In [26]:
print("[OUT]:Count of reorders w.r.t Departments")

reord_depa =merged_all['department'].value_counts().rename_axis('Department Name').reset_index(name="Reorder Count")


alt.Chart(reord_depa).mark_bar().encode(x=alt.X('Department Name',sort=alt.EncodingSortField(
            field="Department Name",  # The field to use for the sort
            order="descending"  # The order to sort in
       )), y='Reorder Count',color='Department Name')

[OUT]:Count of reorders w.r.t Departments


alt.Chart(...)

In [27]:

#merging all tables

all_merged = pd.merge(order_product_train, orders, on='order_id', how='left')
all_merged = pd.merge(all_merged, products, on='product_id', how='left')
all_merged= pd.merge(all_merged, departments, on='department_id', how='left')
all_merged.head()


,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_name,aisle_id,department_id,department
0,1,49302,1,1,112108,train,4,4,10,9.0,Bulgarian Yogurt,120,16,dairy eggs
1,1,11109,2,1,112108,train,4,4,10,9.0,Organic 4% Milk Fat Whole Milk Cottage Cheese,108,16,dairy eggs
2,1,10246,3,0,112108,train,4,4,10,9.0,Organic Celery Hearts,83,4,produce
3,1,49683,4,0,112108,train,4,4,10,9.0,Cucumber Kirby,83,4,produce
4,1,43633,5,1,112108,train,4,4,10,9.0,Lightly Smoked Sardines in Olive Oil,95,15,canned goods


In [0]:
#all dataset as train
data=all_merged.copy()
data_label=data['reordered']
del data['reordered']
del  data['user_id']
del data['product_name']
del data['department']
del data['eval_set']
del data['add_to_cart_order']

In [29]:
#feature colum
print("\n[Out] Making Feature Column:")
Categorical_col = ['order_dow', 'order_hour_of_day','aisle_id', 'department_id','product_id']
Numerical_col = ['order_id', 'order_number','days_since_prior_order']

dow_unique = {value:index for index,value in enumerate(data['order_dow'].unique())}
data['order_dow_cat'] = [float(dow_unique[value]) for value in data.order_dow]
del data['order_dow']

day_hour_unique = {value:index for index,value in enumerate(data['order_hour_of_day'].unique())}
data['order__hour_of_day_cat'] = [float(day_hour_unique[value]) for value in data.order_hour_of_day]
del data['order_hour_of_day']


aisle_unique = {value:index for index,value in enumerate(data['aisle_id'].unique())}
data['aisle_id_cat'] = [float(aisle_unique[value]) for value in data.aisle_id]
del data['aisle_id']

depart_unique = {value:index for index,value in enumerate(data['department_id'].unique())}
data['departmentr_id_cat'] = [float(depart_unique[value]) for value in data.department_id]
del data['department_id']

product_unique = {value:index for index,value in enumerate(data['product_id'].unique())}
data['product_id_cat'] = [float(product_unique[value]) for value in data.product_id]
del data['product_id']

print("\n[Out] Making of  Feature Column done.")

data.head(10)


[Out] Making Feature Column:

[Out] Making of  Feature Column done.


,order_id,order_number,days_since_prior_order,order_dow_cat,order__hour_of_day_cat,aisle_id_cat,departmentr_id_cat,product_id_cat
0,1,4,9.0,0.0,0.0,0.0,0.0,0.0
1,1,4,9.0,0.0,0.0,1.0,0.0,1.0
2,1,4,9.0,0.0,0.0,2.0,1.0,2.0
3,1,4,9.0,0.0,0.0,2.0,1.0,3.0
4,1,4,9.0,0.0,0.0,3.0,2.0,4.0
5,1,4,9.0,0.0,0.0,4.0,1.0,5.0
6,1,4,9.0,0.0,0.0,4.0,1.0,6.0
7,1,4,9.0,0.0,0.0,5.0,0.0,7.0
8,36,23,30.0,1.0,1.0,6.0,0.0,8.0
9,36,23,30.0,1.0,1.0,7.0,3.0,9.0


In [30]:
#split data set as train and test
#data=data[:20000]
#data_label=data_label[:20000]
X_train, X_test, y_train, y_test = train_test_split(data, data_label, test_size=0.30)
X_test,y_test,X_prediction,y_prediction=X_test[100000:],y_test[100000:],X_test[0:100000],y_test[0:100000]

print("[OUT]:length of trainig set: ",len(X_train))
print("[OUT]:length of test set: ",len(X_test))
print("[OUT]:length of trainig  label set: ",len(y_train))
print("[OUT]:length of test label set: ",len(y_test))
print("[OUT]:length of prediction data set: ",len(X_prediction))
print("[OUT]:length of prediction label set: ",len(y_prediction))
X_train.head()

[OUT]:length of trainig set:  969231
[OUT]:length of test set:  315386
[OUT]:length of trainig  label set:  969231
[OUT]:length of test label set:  315386
[OUT]:length of prediction data set:  100000
[OUT]:length of prediction label set:  100000


,order_id,order_number,days_since_prior_order,order_dow_cat,order__hour_of_day_cat,aisle_id_cat,departmentr_id_cat,product_id_cat
30283,73750,28,3.0,1.0,7.0,25.0,2.0,1728.0
1302214,3216800,8,13.0,4.0,15.0,0.0,0.0,2268.0
249588,608872,81,11.0,5.0,10.0,97.0,13.0,12118.0
12947,31241,21,30.0,4.0,2.0,28.0,0.0,5548.0
174474,427475,13,9.0,2.0,5.0,63.0,6.0,10242.0


In [0]:
#model XGB 

model=XGBClassifier(max_depth=10,n_estimators=100, objective='binary:logistic',n_jobs=10)


In [32]:
#train and test 
model.fit(X_train, y_train,eval_set=[(X_train, y_train), (X_test, y_test)],eval_metric='logloss',verbose=True)


[0]	validation_0-logloss:0.677302	validation_1-logloss:0.677699
[1]	validation_0-logloss:0.664361	validation_1-logloss:0.665143
[2]	validation_0-logloss:0.653651	validation_1-logloss:0.654855
[3]	validation_0-logloss:0.644763	validation_1-logloss:0.646302
[4]	validation_0-logloss:0.637062	validation_1-logloss:0.638938
[5]	validation_0-logloss:0.630715	validation_1-logloss:0.632968
[6]	validation_0-logloss:0.625253	validation_1-logloss:0.627822
[7]	validation_0-logloss:0.620525	validation_1-logloss:0.623444
[8]	validation_0-logloss:0.616568	validation_1-logloss:0.619801
[9]	validation_0-logloss:0.61306	validation_1-logloss:0.616625
[10]	validation_0-logloss:0.610004	validation_1-logloss:0.613883
[11]	validation_0-logloss:0.607378	validation_1-logloss:0.611575
[12]	validation_0-logloss:0.605158	validation_1-logloss:0.609647
[13]	validation_0-logloss:0.603131	validation_1-logloss:0.607946
[14]	validation_0-logloss:0.601369	validation_1-logloss:0.606544
[15]	validation_0-logloss:0.599744	v

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=10,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=10,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [33]:
#evaluation result
eval_result = model.evals_result()
print(eval_result)

{'validation_0': {'logloss': [0.677302, 0.664361, 0.653651, 0.644763, 0.637062, 0.630715, 0.625253, 0.620525, 0.616568, 0.61306, 0.610004, 0.607378, 0.605158, 0.603131, 0.601369, 0.599744, 0.598257, 0.597025, 0.595842, 0.594836, 0.593913, 0.593073, 0.59221, 0.591464, 0.590696, 0.590027, 0.589435, 0.588796, 0.588302, 0.587677, 0.587199, 0.586493, 0.586107, 0.585655, 0.585155, 0.584639, 0.584215, 0.583859, 0.583483, 0.583101, 0.582789, 0.582431, 0.582054, 0.5816, 0.581361, 0.580955, 0.580627, 0.58045, 0.5801, 0.579899, 0.579575, 0.579111, 0.578778, 0.578555, 0.578309, 0.578026, 0.577805, 0.577566, 0.577182, 0.576949, 0.576813, 0.576486, 0.57616, 0.575907, 0.575668, 0.575522, 0.575257, 0.574957, 0.574729, 0.57446, 0.57426, 0.574064, 0.573916, 0.573772, 0.573511, 0.573306, 0.573051, 0.572818, 0.572543, 0.572383, 0.572143, 0.571795, 0.571688, 0.571371, 0.571144, 0.570922, 0.570755, 0.570599, 0.570384, 0.570132, 0.570034, 0.569787, 0.569572, 0.569349, 0.569037, 0.568914, 0.568823, 0.568727, 

In [34]:
#checking prediction accuracy on test data

test_pred = model.predict(X_prediction)
predictions_test = [round(val) for val in test_pred]
accuracy = accuracy_score(y_prediction, predictions_test)
print("[OUT]: Accuracy on prediction data is: ",round(accuracy,5))



[OUT]: Accuracy on prediction data is:  0.67958


In [37]:
#Now creating model using Sikit pipline
model_pipeline = Pipeline([('Imputer',Imputer()),('XGBClassifier',XGBClassifier())])
 
#parameter list                     
param_ = {
    "XGBClassifier__n_estimators": [50,100],
    "XGBClassifier__learning_rate": [0.1, 0.5],
    "XGBClassifier__max_depth":[5,10],
}
                       
#parameter for fitting                        
fit_params = {"XGBClassifier__eval_set": [(X_test.values, y_test.values)], 
              "XGBClassifier__early_stopping_rounds": 10, 
              "XGBClassifier__verbose": True}      

#searching best parameter                      
model_grid = GridSearchCV(model_pipeline, cv=5, param_grid=param_)                      

#model fit                     
model_grid.fit(X_train.values,y_train.values, **fit_params)                      

print("[OUT]:Model is now trained.")                       

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:66: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:66: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:66: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


[0]	validation_0-error:0.350992
Will train until validation_0-error hasn't improved in 10 rounds.
[1]	validation_0-error:0.346369
[2]	validation_0-error:0.346296
[3]	validation_0-error:0.345564
[4]	validation_0-error:0.344993
[5]	validation_0-error:0.343116
[6]	validation_0-error:0.342434
[7]	validation_0-error:0.341819
[8]	validation_0-error:0.341252
[9]	validation_0-error:0.341093
[10]	validation_0-error:0.340811
[11]	validation_0-error:0.340687
[12]	validation_0-error:0.340725
[13]	validation_0-error:0.340548
[14]	validation_0-error:0.339362
[15]	validation_0-error:0.339264
[16]	validation_0-error:0.338956
[17]	validation_0-error:0.338401
[18]	validation_0-error:0.337678
[19]	validation_0-error:0.337653
[20]	validation_0-error:0.337402
[21]	validation_0-error:0.336949
[22]	validation_0-error:0.336771
[23]	validation_0-error:0.336654
[24]	validation_0-error:0.336562
[25]	validation_0-error:0.336159
[26]	validation_0-error:0.335931
[27]	validation_0-error:0.335757
[28]	validation_0-er

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:66: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


[0]	validation_0-error:0.349546
Will train until validation_0-error hasn't improved in 10 rounds.
[1]	validation_0-error:0.34758
[2]	validation_0-error:0.345735
[3]	validation_0-error:0.345383
[4]	validation_0-error:0.343116
[5]	validation_0-error:0.341753
[6]	validation_0-error:0.343034
[7]	validation_0-error:0.341474
[8]	validation_0-error:0.341134
[9]	validation_0-error:0.341147
[10]	validation_0-error:0.340925
[11]	validation_0-error:0.340326
[12]	validation_0-error:0.339505
[13]	validation_0-error:0.339048
[14]	validation_0-error:0.339302
[15]	validation_0-error:0.33855
[16]	validation_0-error:0.338331
[17]	validation_0-error:0.338018
[18]	validation_0-error:0.337894
[19]	validation_0-error:0.337361
[20]	validation_0-error:0.337453
[21]	validation_0-error:0.337054
[22]	validation_0-error:0.336809
[23]	validation_0-error:0.336569
[24]	validation_0-error:0.336109
[25]	validation_0-error:0.335846
[26]	validation_0-error:0.335678
[27]	validation_0-error:0.335522
[28]	validation_0-erro

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:66: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


...

[0]	validation_0-error:0.335367
Will train until validation_0-error hasn't improved in 10 rounds.
[1]	validation_0-error:0.333135
[2]	validation_0-error:0.332012
[3]	validation_0-error:0.330199
[4]	validation_0-error:0.329003
[5]	validation_0-error:0.328061
[6]	validation_0-error:0.327947
[7]	validation_0-error:0.327912
[8]	validation_0-error:0.326476
[9]	validation_0-error:0.326463
[10]	validation_0-error:0.326083
[11]	validation_0-error:0.325379
[12]	validation_0-error:0.325278
[13]	validation_0-error:0.325125
[14]	validation_0-error:0.324697
[15]	validation_0-error:0.324323
[16]	validation_0-error:0.324377
[17]	validation_0-error:0.324269
[18]	validation_0-error:0.32413
[19]	validation_0-error:0.324076
[20]	validation_0-error:0.323978
[21]	validation_0-error:0.324117
[22]	validation_0-error:0.324028
[23]	validation_0-error:0.323832
[24]	validation_0-error:0.323879
[25]	validation_0-error:0.323654
[26]	validation_0-error:0.323274
[27]	validation_0-error:0.323166
[28]	validation_0-err

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:66: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


[0]	validation_0-error:0.334894
Will train until validation_0-error hasn't improved in 10 rounds.
[1]	validation_0-error:0.332748
[2]	validation_0-error:0.332193
[3]	validation_0-error:0.331074
[4]	validation_0-error:0.330376
[5]	validation_0-error:0.329308
[6]	validation_0-error:0.328772
[7]	validation_0-error:0.32829
[8]	validation_0-error:0.328226
[9]	validation_0-error:0.328147
[10]	validation_0-error:0.327871
[11]	validation_0-error:0.32686
[12]	validation_0-error:0.326413
[13]	validation_0-error:0.326197
[14]	validation_0-error:0.325855
[15]	validation_0-error:0.325557
[16]	validation_0-error:0.325208
[17]	validation_0-error:0.325236
[18]	validation_0-error:0.325132
[19]	validation_0-error:0.324688
[20]	validation_0-error:0.324529
[21]	validation_0-error:0.324323
[22]	validation_0-error:0.324209
[23]	validation_0-error:0.323959
[24]	validation_0-error:0.323924
[25]	validation_0-error:0.323749
[26]	validation_0-error:0.323816
[27]	validation_0-error:0.323657
[28]	validation_0-erro

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:66: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


[0]	validation_0-error:0.335554
Will train until validation_0-error hasn't improved in 10 rounds.
[1]	validation_0-error:0.333379
[2]	validation_0-error:0.332954
[3]	validation_0-error:0.33225
[4]	validation_0-error:0.331616
[5]	validation_0-error:0.33179
[6]	validation_0-error:0.331185
[7]	validation_0-error:0.330592
[8]	validation_0-error:0.330094
[9]	validation_0-error:0.330113
[10]	validation_0-error:0.329517
[11]	validation_0-error:0.32927
[12]	validation_0-error:0.32926
[13]	validation_0-error:0.329171
[14]	validation_0-error:0.329019
[15]	validation_0-error:0.328737
[16]	validation_0-error:0.328702
[17]	validation_0-error:0.328347
[18]	validation_0-error:0.328211
[19]	validation_0-error:0.328252
[20]	validation_0-error:0.328207
[21]	validation_0-error:0.327776
[22]	validation_0-error:0.327342
[23]	validation_0-error:0.327142
[24]	validation_0-error:0.326815
[25]	validation_0-error:0.326796
[26]	validation_0-error:0.326371
[27]	validation_0-error:0.326191
[28]	validation_0-error:

In [39]:
print("[OUT]: Best parameter list : ",model_grid.best_params_)
print("[OUT]: Model Score on prediction data: ",round(model_grid.score(X_prediction.values,y_prediction),5))

[OUT]: Best parameter list :  {'XGBClassifier__learning_rate': 0.1, 'XGBClassifier__max_depth': 10, 'XGBClassifier__n_estimators': 100}
[OUT]: Model Score on prediction data:  0.67963
